In [1]:
import datetime
print(datetime.datetime.now())

2022-01-07 16:38:16.815115


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


In [3]:
grid_data_file = os.path.join(os.getcwd().split('data')[0], 'bi.pickle')
print(grid_data_file)
with open(grid_data_file,'rb') as f:
    load_array= pickle.load(f)
exc = load_array[0]
inh = load_array[1]
print(len(exc))

/home/lena/Promotion/neurolib/GUI/current/gui/bi.pickle
147


In [4]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [5]:
d_array = np.arange(500., 20, -0.1)
t0_array = d_array - 20.
t_pen = np.zeros(( t0_array.shape ))
for i in range(len(t_pen)):
    t_pen[i] = t0_array[i] / d_array[i]

In [6]:
bestControl_ = [None] * len(exc) 
bestState_ = [None] * len(exc) 
cost_ = [None] * len(exc) 
costnode_ = [None] * len(exc) 
weights_ = [None] * len(exc) 
convergence_ = [None] * len(exc) 

initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)
ws = [None] * len(exc)
conv = [None] * len(exc)
conv_shift = [None] * len(exc)

In [7]:
file_ = '1_E.pickle'
folder = str(os.getcwd().split('data_2')[1])
print(folder)
case = str(folder[1])

/1/runtime


In [8]:
dur_pre = 10
dur_post = 10

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-10
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

c_var = [1]
p_var = [0]

if case in ['1', '2']:    # low to high
    max_I = [3., -3.]
else:
    max_I = [-3., 3.]
    
if case in ['1', '3']:    # sparsity
    factor_ws = 1.
    factor_we = 0.
else:
    factor_ws = 0.
    factor_we = 1.
    
maxC = [5., -5., 0.18, 0.]
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [9]:
if os.path.isfile(file_) :
    print("file found")
    
    with open(file_,'rb') as f:
        load_array = pickle.load(f)

    bestControl_ = load_array[0]
    bestState_ = load_array[1]
    cost_ = load_array[2]
    costnode_ = load_array[3]
    weights_ = load_array[4]
    

In [10]:
# get initial parameters and target states

i_range = [20]

data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = 3000.
    
    control0 = aln.getZeroControl()
    control0 = functions.step_control(aln, maxI_ = max_I[0])

    aln.run(control=control0)
    
    target_rates = np.zeros((2))
    target_rates[0] = aln.rates_exc[0,-1] 
    target_rates[1] = aln.rates_inh[0,-1]

    control0 = functions.step_control(aln, maxI_ = max_I[1])
    aln.run(control=control0)

    init_state_vars = np.zeros(( len(state_vars) ))
    for j in range(len(state_vars)):
        if aln.state[state_vars[j]].size == 1:
            init_state_vars[j] = aln.state[state_vars[j]][0]
        else:
            init_state_vars[j] = aln.state[state_vars[j]][0,-1]

    initVars[i] = init_state_vars
    target[i] = [target_rates[0], target_rates[1]]

-------  20 0.4500000000000001 0.4750000000000002


In [11]:
# get uncontrolled cost

data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    dur = d_array[0]
    aln.params.duration = dur
    
    target_ = aln.getZeroTarget()
    target_[:,0,:] = target[i][0]
    target_[:,1,:] = target[i][1]
            
    cost.setParams(1.0, 0.0, 0.0)

    setinit(initVars[i], aln)
    control0 = aln.getZeroControl()

    # "HS", "FR", "PR", "HZ"
    cgv = None
    max_it = 0

    bestControl_init_, bestState_init_, cost_init_, runtime_init_, grad_init_, phi_init_, phi1_, costnode_init_ = aln.A1(
        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
        prec_variables_ = p_var, transition_time_ = t_pen[0])
    
    cost_uncontrolled[i] = cost_init_[0]

-------  20 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 0.0
interpolate adjoint :  True True True
initial cost =  316931.9421284
RUN  0 , total integrated cost =  316931.9421284
Gradient descend method:  None
RUN  0 , total integrated cost =  316931.9421284
Improved over  0  iterations in  0.0  seconds by  0.0  percent.


In [12]:
cgv = None
j = 0

wp = 1.

for i in i_range:

    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
        
    found_exponent = False
    found_digit_0 = False
    found_digit_1 = False
    exponent = 5
    sig_digit = [1., 0.]
    
    while not found_digit_1:
        
        ws[i] = sig_digit[0] * 10**exponent + sig_digit[1] * 10**(exponent-1.)
    
        dur = d_array[0]
        aln.params.duration = dur
        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        cost.setParams(wp, 0., ws[i])
        max_it = int(10)
        if found_digit_0:
            max_it = int(50)

        bestControl_[i], bestState_[i], cost_[i], runtime_, grad_, phi_, phi1_, costnode_[i] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[0])
        
        if found_exponent == False and cost_[i][-1] == 0.:
            exponent -= 1
            continue
        elif found_exponent == False and found_digit_0 == False:
            found_exponent = True
            sig_digit[0] = 9.
        elif found_exponent == True and found_digit_0 == False and cost_[i][-1] == 0.:
            sig_digit[0] -= 1.
        elif found_exponent == True and found_digit_0 == False:
            found_digit_0 = True
            sig_digit[1] = 9.
        elif found_exponent == True and found_digit_0 == True and found_digit_1 == False and cost_[i][-1] == 0.:
            sig_digit[1] -= 1.
        elif found_exponent == True and found_digit_0 == True and found_digit_1 == False:
            print("sparsity weight = ", ws[i])

            break             

-------  20 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 100000.0
interpolate adjoint :  True True True
initial cost =  316931.9421284
RUN  0 , total integrated cost =  316931.9421284
Gradient descend method:  None
RUN  1 , total integrated cost =  316931.9421284
Control only changes marginally.
RUN  1 , total integrated cost =  316931.9421284
Improved over  1  iterations in  9.15832148696063  seconds by  0.0  percent.
set cost params:  1.0 0.0 10000.0
interpolate adjoint :  True True True
initial cost =  316931.9421284
RUN  0 , total integrated cost =  316931.9421284
Gradient descend method:  None
RUN  1 , total integrated cost =  90650.31567479047
RUN  2 , total integrated cost =  61428.08733185488
RUN  3 , total integrated cost =  49082.332360617955
RUN  4 , total integrated cost =  39925.530798281536
RUN  5 , total integrated cost =  39822.14012570088
RUN  6 , total integrated cost =  39748.83135873175
RUN  7 , total integrated cost =  39666.098363872676
RUN  8 ,

In [13]:
c_var = [0,1]

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    k = 0
    
    while not conv[i]:
        max_it = int(100 + 100*k)

        weight_ = ( cost_uncontrolled[i] - costnode_[i][0][0][0] ) / sum( costnode_[i][2][0][:] )
            
        dur = d_array[0]
        aln.params.duration = dur
        cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
        weights_[i] = cost.getParams()

        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        control0 = bestControl_[i][:,:,n_pre-1:-n_post+1]

        bestControl_[i], bestState_[i], cost_[i], runtime_, grad_, phi_, phi1_, costnode_[i] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[0])
        
        if cost_[i][2] == 0.:
            conv[i] = True

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)
        
        k += 1

-------  20 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 87917.21203962908
interpolate adjoint :  True True True
initial cost =  316931.9421284
RUN  0 , total integrated cost =  316931.9421284
Gradient descend method:  None
RUN  1 , total integrated cost =  316473.57318635925
RUN  2 , total integrated cost =  316325.95221700036
RUN  3 , total integrated cost =  316163.5378958451
RUN  4 , total integrated cost =  316014.9057912519
RUN  5 , total integrated cost =  315863.064043494
RUN  6 , total integrated cost =  315779.6846496601
RUN  7 , total integrated cost =  315684.7098085229
RUN  8 , total integrated cost =  315608.30667571304
RUN  9 , total integrated cost =  315518.3233776793
RUN  10 , total integrated cost =  315455.218264337
RUN  11 , total integrated cost =  315387.4939208968
RUN  12 , total integrated cost =  315329.9992937647
RUN  13 , total integrated cost =  315261.1213928859
RUN  14 , total integrated cost =  315213.0787460618
RUN  15 , total integra

RUN  90 , total integrated cost =  316843.1549326932
RUN  100 , total integrated cost =  316840.8635841287
RUN  110 , total integrated cost =  316837.15336449863
RUN  120 , total integrated cost =  316832.8680132174
RUN  130 , total integrated cost =  316673.92104730464
RUN  140 , total integrated cost =  316666.8542633652
RUN  150 , total integrated cost =  316644.81343964615
RUN  160 , total integrated cost =  316628.8274795376
RUN  170 , total integrated cost =  316625.96099182713
RUN  180 , total integrated cost =  316623.84142049577
RUN  190 , total integrated cost =  316618.98763390735
RUN  200 , total integrated cost =  316614.80633940717
RUN  300 , total integrated cost =  316273.2085715671
RUN  400 , total integrated cost =  316139.20484226797
RUN  400 , total integrated cost =  316139.20484226797
Improved over  400  iterations in  92.92107364500407  seconds by  0.25012855466960104  percent.
set cost params:  1.0 0.0 94450.04878867518
interpolate adjoint :  True True True
init

RUN  200 , total integrated cost =  316850.76909648156
RUN  300 , total integrated cost =  316798.4397972309
Control only changes marginally.
RUN  382 , total integrated cost =  316764.1821784378
Improved over  382  iterations in  87.30666269297944  seconds by  0.05293248412753826  percent.
set cost params:  1.0 0.0 94729.722680239
interpolate adjoint :  True True True
initial cost =  316931.9421284
RUN  0 , total integrated cost =  316931.9421284
Gradient descend method:  None
RUN  1 , total integrated cost =  316931.94211956643
RUN  2 , total integrated cost =  316931.9419535702
RUN  3 , total integrated cost =  316931.9230421685
RUN  4 , total integrated cost =  316931.88865336636
RUN  5 , total integrated cost =  316931.88846260484
RUN  6 , total integrated cost =  316931.88843414973
RUN  7 , total integrated cost =  316931.88843143225
RUN  8 , total integrated cost =  316931.88842812047
RUN  9 , total integrated cost =  316931.8884277134
RUN  10 , total integrated cost =  316931.8

In [14]:
#for i in i_range:
for i in i_range:
        
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = dur
    max_it = int(0)
    
    for l in range( bestControl_[i].shape[2] ):
        if np.abs(bestControl_[i][0,0,l]) > 5. * 1e-6 or np.abs(bestControl_[i][0,1,l]) > 5. * 1e-6:
            init_ind = l - n_pre
            break
            
    if init_ind == -1:
        init_ind = 0
            
    print("init ind = ", init_ind)

    weight_ = 1.
    cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
    weights_[i] = cost.getParams()

    setinit(initVars[i], aln)

    target_ = aln.getZeroTarget()
    target_[:,0,:] = target[i][0]
    target_[:,1,:] = target[i][1]

    control0 = aln.getZeroControl()
    if init_ind != 0:
        control0[:,:,:-init_ind] = bestControl_[i][:,:,n_pre-1+init_ind:-n_post+1]
    else:
        control0 = bestControl_[i][:,:,n_pre-1:-n_post+1]

    bestControl_[i], bestState_[i], cost_[i], runtime_, grad_, phi_, phi1_, costnode_[i] = aln.A1(
        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
        prec_variables_ = p_var, transition_time_ = t_pen[0])

-------  20 0.4500000000000001 0.4750000000000002
init ind =  1744
set cost params:  1.0 0.0 1.0
interpolate adjoint :  True True True
initial cost =  3.34312946226113
RUN  0 , total integrated cost =  3.34312946226113
Gradient descend method:  None
RUN  0 , total integrated cost =  3.34312946226113
Improved over  0  iterations in  0.0  seconds by  0.0  percent.


In [15]:
conv_shift = [None] * len(exc)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
        
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = dur
    max_it = int(1000)
    k = 0
    
    while not conv_shift[i]:
            
        weight_ = 1.
        cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
        weights_[i] = cost.getParams()

        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        control0 = bestControl_[i][:,:,n_pre-1:-n_post+1]

        bestControl_[i], bestState_[i], cost_[i], runtime_, grad_, phi_, phi1_, costnode_[i]= aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[0])
                
        if cost_[i][-1] == 0.:
            conv_shift[i] = True

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)
        
        k += 1

-------  20 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 1.0
interpolate adjoint :  True True True
initial cost =  3.34312946226113
RUN  0 , total integrated cost =  3.34312946226113
Gradient descend method:  None
RUN  1 , total integrated cost =  3.3314107122611323
RUN  2 , total integrated cost =  3.3311818304251926
RUN  3 , total integrated cost =  3.3311686709846215
RUN  4 , total integrated cost =  3.331168664189128
RUN  5 , total integrated cost =  3.331168664189128
Control only changes marginally.
RUN  5 , total integrated cost =  3.331168664189128
Improved over  5  iterations in  0.8430651659728028  seconds by  0.35777250647997505  percent.


In [16]:
print(datetime.datetime.now())

2022-01-07 16:48:26.795984


In [28]:
data.set_parameters(aln)

i = 20
print("------- ", i, exc[i], inh[i])
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.

dur = d_array[0]
aln.params.duration = dur

target_ = aln.getZeroTarget()
target_[:,0,:] = target[i][0]
target_[:,1,:] = target[i][1]

cost.setParams(1.0, 0.0, 0.0)

setinit(initVars[i], aln)
control0 = aln.getZeroControl()
control0 = bestControl_init_[:,:,100:-100]

# "HS", "FR", "PR", "HZ"
cgv = None
max_it = 1

%prun bestControl_init_, bestState_init_, cost_init_, runtime_init_, grad_init_, phi_init_, phi1_, costnode_init_ = aln.A1(        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,        prec_variables_ = p_var, transition_time_ = t_pen[0])

-------  20 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 0.0
interpolate adjoint :  True True True
initial cost =  16.24806421013515
RUN  0 , total integrated cost =  16.24806421013515
Gradient descend method:  None
RUN  1 , total integrated cost =  12.655242367918078
RUN  1 , total integrated cost =  12.655242367918078
Improved over  1  iterations in  0.1566801139852032  seconds by  22.112307015477924  percent.
 

In [29]:
step_time = [0.049, 0.110, 0.098, 0.098, 0.105, 0.105]
tot_time = [0.114 , 0.176 , 0.165 , 0.163, 0.168, 0.169]

In [30]:
for i in range(len(step_time)):
    print(step_time[i]/tot_time[i])

0.4298245614035088
0.625
0.593939393939394
0.6012269938650306
0.6249999999999999
0.621301775147929
